## 6. Prediction - Long-Short Term Memory Model

### About this Method and the assumption we have behind it:
- LSTM Model is a type of recurrent neural networks (RNN)
- designed to handle sequence precition problems, particularly those with long-range dependencies. 

The Solution here somehow goes from 6 or 7 in 7 or 8 steps onwards.. i suggest this lies on the randomness in all the data, that the number which the model picks seem to be nearly distributed


In [249]:
# Import Libraries needed
import pandas as pd
import matplotlib.pyplot as plt
import ast
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
from keras.layers import Input
from keras.callbacks import EarlyStopping

plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')

### Load the Data we Stored in the data Folder with Notebook 1

In [250]:

# Load the CSV file into a DataFrame
frequency_data = pd.read_csv("data/frequency_data.csv")


### Convert date from String to Datetime for Evaluation
And further Data Processing for our needs

In [251]:
# convert the string to datetime
frequency_data["date"] = pd.to_datetime(frequency_data["date"], format='%d.%m.%Y')

# Convert the Lottozahl column to a list of integers
frequency_data["Lottozahl"] = frequency_data["Lottozahl"].apply(lambda x: eval(x))

# Expand the Lottozahl column into seperate columns
df_expanded = pd.DataFrame(frequency_data["Lottozahl"].tolist(), columns=['Lottozahl_1', 'Lottozahl_2', 'Lottozahl_3', 'Lottozahl_4', 'Lottozahl_5', 'Lottozahl_6'])

# Create the final Dataframe
df_final = pd.concat([frequency_data.drop(columns= "Lottozahl"), df_expanded], axis=1)


# Cast the Date into own columns
df_final["day"] = df_final["date"].dt.day
df_final["month"] = df_final["date"].dt.month
df_final["year"] = df_final["date"].dt.year

# Drop the Superzahl Column (we only Predict the 6 Lotto Numbers)
df_final.drop(["Superzahl", "id","date"], axis=1, inplace=True)

df_final.head()

,Lottozahl_1,Lottozahl_2,Lottozahl_3,Lottozahl_4,Lottozahl_5,Lottozahl_6,day,month,year
0,3,12,13,16,23,41,9,10,1955
1,3,12,18,30,32,49,16,10,1955
2,12,14,23,24,34,36,23,10,1955
3,4,13,23,30,36,44,30,10,1955
4,5,6,31,39,44,49,6,11,1955


### Scale the Features for the LSTM Model 
- we will use a MinMax Scaler here and restrict the range from 0-1 due to we have only positive values

In [252]:
dataset = df_final.values

In [253]:
X = dataset[:, 6:]  # Input features: Lotto numbers + date components
y = dataset[:, :6]  # Output labels: Future lottery numbers

In [254]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [255]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2)

In [256]:
time_step = 1

In [257]:
# Reshape input data for LSTM (samples, time steps, features)
X_train = X_train.reshape(X_train.shape[0], time_step, X_train.shape[1])
X_test = X_test.reshape((X_test.shape[0], time_step, X_test.shape[1]))

In [258]:
# Define the LSTM model with dropout layers
model = Sequential()
model.add(Input(shape=(1, 3)))  # Input layer with the specified shape
model.add(LSTM(50, return_sequences=True))  # First LSTM layer
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Dense(6))  # Output layer with 9 neurons for the next sequence of lottery numbers


In [259]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=0,restore_best_weights=True)

In [260]:
model.compile(optimizer="adam", loss="mse")

In [261]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 830.0404 - val_loss: 807.7285
Epoch 2/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 797.9586 - val_loss: 757.0430
Epoch 3/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 728.4808 - val_loss: 661.9863
Epoch 4/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 631.5504 - val_loss: 543.6807
Epoch 5/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 515.9481 - val_loss: 431.1460
Epoch 6/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 406.6402 - val_loss: 337.8867
Epoch 7/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 321.1883 - val_loss: 264.8910
Epoch 8/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 247.9281 - val_loss: 210.0337
Epoch 9/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 197.1795 - val_loss: 169.0596
Epoch 10/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 159.2726 - val_loss: 138.7754
Epoch 11/1000
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 133.4868 - val_loss: 116.4417
Epoch 12

In [262]:
# Model evaluation
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Predict the next sequence of lottery numbers and date
predictions = model.predict(X_test)



30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - loss: 50.0734
Test Loss: 50.78819274902344
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step


# Add the New Date in X_new here!! To get your newest prediction

In [263]:
# Assume X_new is your new data with shape (1, 3)
X_new = np.array([27, 4, 2024]).reshape(1, -1)  # Reshape the new data
X_new_scaled = scaler.transform(X_new)  # Scale the new data
X_new_reshaped = X_new_scaled.reshape((1, 1, 3))  # Reshape the new data
predictions = model.predict(X_new_reshaped)  # Make predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step


# Predict your new Numbers for the Week!

In [248]:
# Convert the predicted array to a list of integers
predicted_numbers_list = [int(round(number)) for sublist in predictions[-1] for number in sublist]

print("Predicted Lotto Numbers as List:", predicted_numbers_list)

Predicted Lotto Numbers as List: [7, 14, 22, 29, 36, 44]


# Predictions for the 3 Lottery Ticket Field!

### DISCLAIMER:
- This is a Fun Project to Showcase some IT Skills
- Lottery is a total statistically Random Game
- Do not use or let you inspire by this Prediction!
- Gambling can make you addicted! Only Play with Caution!
- I claim no rights, that my outputs would even be near the drawn value

### About the Predictions:
- In Field 3 we go for the longest Time no See Numbers each Lottery Ticket we play
- The Superzahl only appears once per Lottery Ticket - we will take Random 7 or 4 for each Lottery Ticket
- This is a common Lottery Strategy - even if you win with this Numbers, you would have to share the Prize Pool with many other People
- We will do more complex Prediction Strategies in later Notebooks
- This Prediction will vary more from Week to Week than the first 2, because when a Number gets drawn it refreshes its date and will not show up here for a longer time

In [11]:
# Format the first 6 Numbers for better Output
formatted_numbers = ", ".join(map(str, least_recently_drawn.index))



# Print your Lucky Numbers
print("Your Lucky Numbers For Field 3 (Longest Time not Drawn):",formatted_numbers," Superzahl:", np.random.choice([4,7]))

NameError: name 'least_recently_drawn' is not defined